In [1]:
#Base packages
import os
import pickle

import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

c:\Users\agust\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\agust\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#NLP and text packages

#import re
import spacy
#nlp = spacy.blank("en") # only tokenizes and creates stop word boolean
nlp = spacy.load("en_core_web_sm") #adds lemmas and POS too

Import datasets

In [3]:
nyt80 = pd.read_csv('../1_nyt_ineq_articles/nyt80_merge.csv')
congress80 = pd.read_csv('../1_congress_data_proc/congress80_merge.csv')

In [4]:
len(congress80)

3777

In [5]:
len(nyt80)

10681

Tokenize

In [6]:
nlp = spacy.load("en_core_web_sm") 

In [7]:
# Test text
#text = "#The %quick > brown-fox hospitals camila's dog is going to Marlon jum!ps organizer over' 34 case1 hospital organizations the lazy dog."
text ='The World; Brazil Topples Off the Verge Tall and tanned and young and lovely The girl from Ipanema goes walking And when she passes Each one she passes Goes "Aaah"'

# Process the text
doc = nlp(text)

print([token.text for token in doc if token.is_alpha])
print()

# Inspect tokens
for token in doc:
    print(f"Text: {token.text}, Lemma: {token.lemma_}, POS: {token.pos_}, Is stop word: {token.is_stop}")

lemma_tokens_no_stop_words = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
lemma_tokens_no_stop_words


['The', 'World', 'Brazil', 'Topples', 'Off', 'the', 'Verge', 'Tall', 'and', 'tanned', 'and', 'young', 'and', 'lovely', 'The', 'girl', 'from', 'Ipanema', 'goes', 'walking', 'And', 'when', 'she', 'passes', 'Each', 'one', 'she', 'passes', 'Goes', 'Aaah']

Text: The, Lemma: the, POS: DET, Is stop word: True
Text: World, Lemma: World, POS: PROPN, Is stop word: False
Text: ;, Lemma: ;, POS: PUNCT, Is stop word: False
Text: Brazil, Lemma: Brazil, POS: PROPN, Is stop word: False
Text: Topples, Lemma: topple, POS: VERB, Is stop word: False
Text: Off, Lemma: off, POS: ADP, Is stop word: True
Text: the, Lemma: the, POS: DET, Is stop word: True
Text: Verge, Lemma: Verge, POS: PROPN, Is stop word: False
Text: Tall, Lemma: Tall, POS: PROPN, Is stop word: False
Text: and, Lemma: and, POS: CCONJ, Is stop word: True
Text: tanned, Lemma: tanned, POS: ADJ, Is stop word: False
Text: and, Lemma: and, POS: CCONJ, Is stop word: True
Text: young, Lemma: young, POS: ADJ, Is stop word: False
Text: and, Lemma: a

['World',
 'Brazil',
 'topple',
 'Verge',
 'Tall',
 'tanned',
 'young',
 'lovely',
 'girl',
 'Ipanema',
 'goes',
 'walk',
 'pass',
 'pass',
 'go',
 'Aaah']

In [8]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def process_text_for_stm(text):
    # Process text with spaCy
    doc = nlp(text)

    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
    
    return lemmas

In [9]:
def process_text_column(df, text_column="text"):
    """
    Processes the text column of a DataFrame, tokenizing and lemmatizing text,
    excluding stopwords, and updating the DataFrame with the results.
    
    Args:
    - df: DataFrame containing the text column
    - text_column: Name of the column containing text to be processed
    
    Returns:
    - df: Updated DataFrame with 'lemma_tokens_no_stop_words' columns
    """
    lemma_tokens_no_stop_words = []
    
    # Use tqdm to process the text column
    for idx, text in tqdm(enumerate(df[text_column]), desc="Processing text column"):
        try:
            lemma_tokens = process_text_for_stm(text)
            lemma_tokens_no_stop_words.append(lemma_tokens)
        except Exception as e:
            print(f"Error processing row {idx}: {text}")
            print(f"Error message: {e}")
            lemma_tokens_no_stop_words.append(None)
    
    # Add the results as new columns to the DataFrame
    df['lemma_tokens_no_stop_words'] = lemma_tokens_no_stop_words
    
    return df

In [10]:
len(nyt80)

10681

In [11]:
nyt80_tokens = process_text_column(nyt80)

Processing text column: 0it [00:00, ?it/s]

Processing text column: 10681it [1:01:08,  2.91it/s]


In [12]:
congress80_tokens = process_text_column(congress80)

Processing text column: 3777it [10:43,  5.87it/s]


Combine to implement in R

In [13]:
combined_df = pd.concat([nyt80_tokens, congress80_tokens], ignore_index=True)

In [14]:
combined_df['lemma_tokens_no_stop_words']

0        [get, zero, sum, society, distribution, possib...
1        [fed, monetary, policy, credibility, gap, econ...
2        [egypt, move, help, economy, cause, confusion,...
3        [books, times, books, times, edward, cowan, ze...
4        [john, paul, voice, peg, pope, john, paul, ii,...
                               ...                        
14453    [president, holiday, season, approach, appropr...
14454    [speaker, thank, sister, cori, bush, yield, sp...
14455    [speaker, congresswoman, ramirez, come, congre...
14456    [speaker, yield, time, consume, speaker, measu...
14457    [president, rise, today, honor, hugh, espey, i...
Name: lemma_tokens_no_stop_words, Length: 14458, dtype: object

In [15]:
# Step 3: Reformat tokens for STM (join tokens into a string)
combined_df['tokens_R'] = combined_df['lemma_tokens_no_stop_words'].apply(lambda x: ' '.join(x))

# Now, combined_df is ready for the prepDocuments function
combined_df.tail()

,id_,text,author,date_d-m-Y,year,source,party,dataset,lemma_tokens_no_stop_words,tokens_R
14453,CREC-2024-12-11-pt1-PgS6963-4-1,"Mr. President, as the holiday season approache...",Mr. SANDERS,2024-12-11,2024,Independent,Independent,Congress,"[president, holiday, season, approach, appropr...",president holiday season approach appropriate ...
14454,CREC-2024-12-12-pt1-PgH7130-2,"Mr. Speaker, I thank my sister Cori Bush for y...",Mr. BOWMAN,2024-12-12,2024,Democrat,Democrat,Congress,"[speaker, thank, sister, cori, bush, yield, sp...",speaker thank sister cori bush yield speaker w...
14455,CREC-2024-12-12-pt1-PgH7130-6,"Mr. Speaker, Congresswoman Ramirez came into C...",Ms. BUSH,2024-12-12,2024,Democrat,Democrat,Congress,"[speaker, congresswoman, ramirez, come, congre...",speaker congresswoman ramirez come congress li...
14456,CREC-2024-12-16-pt1-PgH7151-10,"Mr. Speaker, I yield myself such time as I may...",Mr. STEIL,2024-12-16,2024,Republican,Republican,Congress,"[speaker, yield, time, consume, speaker, measu...",speaker yield time consume speaker measure hal...
14457,CREC-2024-12-20-pt1-PgS7321-0,"Mr. President, I rise today to honor Hugh Espe...",Mr. SANDERS,2024-12-20,2024,Independent,Independent,Congress,"[president, rise, today, honor, hugh, espey, i...",president rise today honor hugh espey iowa cit...


In [16]:
combined_df = combined_df.rename(columns={'date_d-m-Y': 'date'})
combined_df['year_n'] = combined_df['year'] - 1979

In [17]:
combined_df.to_csv('articles_speeches_dataset_r.csv', index=False)

In [22]:
combined_df

,id_,text,author,date,year,source,party,dataset,lemma_tokens_no_stop_words,tokens_R,year_n
0,0,WHO GETS WHAT THE ZERO-SUM SOCIETY Distributi...,MARC F. PLATTNER; Marc F. Plattner is on the s...,1980-06-08,1980,nyt,Democrat,nyt,"[get, zero, sum, society, distribution, possib...",get zero sum society distribution possibilitie...,1
1,1,FED MONETARY POLICY:CREDIBILITY GAP; Economic ...,ROBERT A. BENNETT,1980-06-14,1980,nyt,Democrat,nyt,"[fed, monetary, policy, credibility, gap, econ...",fed monetary policy credibility gap economic a...,1
2,2,EGYPT'S MOVES TO HELP ECONOMY CAUSING CONFUSIO...,CHRISTOPHER S. WREN,1980-06-23,1980,nyt,Democrat,nyt,"[egypt, move, help, economy, cause, confusion,...",egypt move help economy cause confusion campai...,1
3,3,BOOKS OF THE TIMES; Books Of The Times By Edw...,NaN,1980-07-01,1980,nyt,Democrat,nyt,"[books, times, books, times, edward, cowan, ze...",books times books times edward cowan zero sum ...,1
4,4,'John Paul Is Our Voice' Those who pegged Pop...,NaN,1980-07-13,1980,nyt,Democrat,nyt,"[john, paul, voice, peg, pope, john, paul, ii,...",john paul voice peg pope john paul ii conserva...,1
...,...,...,...,...,...,...,...,...,...,...,...
14453,CREC-2024-12-11-pt1-PgS6963-4-1,"Mr. President, as the holiday season approache...",Mr. SANDERS,2024-12-11,2024,Independent,Independent,Congress,"[president, holiday, season, approach, appropr...",president holiday season approach appropriate ...,45
14454,CREC-2024-12-12-pt1-PgH7130-2,"Mr. Speaker, I thank my sister Cori Bush for y...",Mr. BOWMAN,2024-12-12,2024,Democrat,Democrat,Congress,"[speaker, thank, sister, cori, bush, yield, sp...",speaker thank sister cori bush yield speaker w...,45
14455,CREC-2024-12-12-pt1-PgH7130-6,"Mr. Speaker, Congresswoman Ramirez came into C...",Ms. BUSH,2024-12-12,2024,Democrat,Democrat,Congress,"[speaker, congresswoman, ramirez, come, congre...",speaker congresswoman ramirez come congress li...,45
14456,CREC-2024-12-16-pt1-PgH7151-10,"Mr. Speaker, I yield myself such time as I may...",Mr. STEIL,2024-12-16,2024,Republican,Republican,Congress,"[speaker, yield, time, consume, speaker, measu...",speaker yield time consume speaker measure hal...,45
